# Neural Network Model (CNNRNN)

In [88]:
import numpy as np
import tensorflow as tf
import pickle
import gensim
import os, shutil, time
import model.CNNRNN as CNNRNN
from importlib import reload
from lime.lime_text import LimeTextExplainer
import pandas as pd

In [2]:
# Load word2vec pre-trained model
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)
word2vec_matrix = word2vec_model.syn0

/home/mr_ham/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
# Add <UNK> and <PAD> vectors to word2vec matrix
unk_vec = np.random.uniform(-0.25,0.25,300) 
pad_vec = np.zeros(300) 

word2vec_matrix = np.vstack((word2vec_matrix, unk_vec))
word2vec_matrix = np.vstack((word2vec_matrix, pad_vec))

word2vec_words = [k for k in word2vec_model.vocab.keys()]
word2vec_words = word2vec_words + ['<unk>', '<pad>']

id_to_word = dict(enumerate(word2vec_words))
word_to_id = {v:k for k,v in id_to_word.items()}

In [48]:
def input_generator(data, max_sent, max_word):
    
    # This function takes the input data in tokens and converted into matrix with indices
    
    inputs_x = np.zeros([len(data), max_sent, max_word])
    inputs_y = np.zeros([len(data), 2])
    num_doc = 0
    
    for i in data.index:
        inputs_y[num_doc] = [0, 1] if data.loc[i, 'y'] == 1 else [1, 0]
        
        inputs_x_sent = np.zeros([max_sent, max_word])
        inputs_x_sent.fill(3000001)
        
        sents = ' '.join(data.loc[i, 'text']).split('</s>')
        num_sent = 0
        
        for s in sents:
            if num_sent < max_sent:
                if len(s) > 0:
                    my_sent = s.rstrip().lstrip().split()
                    num_word = 0
                    for w in my_sent:
                        if num_word < max_word:
                            inputs_x_sent[num_sent, num_word] = word_to_id.get(w)
                            num_word += 1
                        else:
                            break

                    num_sent += 1
            else:
                break

        inputs_x[num_doc] = inputs_x_sent
        
        num_doc += 1
    
    return inputs_x, inputs_y

def batch_generator(inputs_x, inputs_y, batch_size):
    num_batch = (len(inputs_y)) // batch_size
    for i in range(num_batch):
        yield inputs_x[i*batch_size:(i+1)*batch_size], inputs_y[i*batch_size:(i+1)*batch_size]
        
def score_dataset(lm, session, inputs_x, inputs_y, batch_size, name="Train"):
    # For scoring, we can use larger batches to speed things up.
    bi = batch_generator(inputs_x, inputs_y, batch_size)
    cost, pred_accuracy = run_epoch(lm, session, bi, 
                                    learning_rate=0.0, train=False, 
                                    verbose=True, tick_s=60)
    print("{:s}: avg. loss: {:.03f}  (perplexity: {:.02f}), accuracy: {:.03f}".format(name, cost, np.exp(cost), pred_accuracy))
    return cost, pred_accuracy

def pretty_timedelta(fmt="%d:%02d:%02d", since=None, until=None):
    """Pretty-print a timedelta, using the given format string."""
    since = since or time.time()
    until = until or time.time()
    delta_s = until - since
    hours, remainder = divmod(delta_s, 3600)
    minutes, seconds = divmod(remainder, 60)
    return fmt % (hours, minutes, seconds)

In [5]:
# Load data
train_data = pickle.load(open("train.p", "rb"))
dev_data = pickle.load(open("dev.p", "rb"))
test_data = pickle.load(open("test.p", "rb"))

In [6]:
# Specify the maximum sentence length per doc 
# and maximum number of words per sentence

max_sent_length = 200
sequence_length = 100

train_x, train_y = input_generator(train_data, max_sent_length, sequence_length)
dev_x, dev_y = input_generator(dev_data, max_sent_length, sequence_length)
test_x, test_y = input_generator(test_data, max_sent_length, sequence_length)

In [7]:
print(train_x.shape)
print(train_y.shape)
print(dev_x.shape)
print(dev_y.shape)
print(test_x.shape)
print(test_y.shape)

(17052, 200, 100)
(17052, 2)
(2131, 200, 100)
(2131, 2)
(2133, 200, 100)
(2133, 2)


In [52]:
def run_epoch(lm, session, batch_iterator,
              train=False, verbose=False,
              tick_s=10, learning_rate=None):
    
    assert(learning_rate is not None)
    start_time = time.time()
    tick_time = start_time  # for showing status
    total_cost = 0.0  # total cost, summed over all words
    total_accuracy = 0.0
    total_batches = 0
    total_words = 0

    if train:
        train_op = lm.train_step_
    else:
        train_op = tf.no_op()
    
    loss = lm.loss_ 
    accuracy = lm.accuracy_

    for i, (x, y) in enumerate(batch_iterator):
        cost = 0.0
        pred_accuracy = 0.0
        
        feed_dict = {lm.input_x_ : x,
                     lm.input_y_ : y,
                     lm.learning_rate_ : learning_rate}
        
        cost, pred_accuracy, _ = session.run([loss, accuracy, train_op], feed_dict=feed_dict)

        total_cost += cost
        total_accuracy += pred_accuracy
        total_batches = i + 1

        # Print average loss-so-far for epoch
        if verbose and (time.time() - tick_time >= tick_s):
            avg_cost = total_cost / total_batches
            avg_accuracy = total_accuracy / total_batches
            print("[batch {:d}]: loss = {:.3f}, accuracy = {:.3f}".format(i, avg_cost, avg_accuracy))
            tick_time = time.time()  # reset time ticker
        
    return (total_cost / total_batches, total_accuracy / total_batches)

In [38]:
# Training parameters
max_time = 25
batch_size = 200
learning_rate = 0.01
num_epochs = 2

# Model parameters
model_params = dict(max_sent_length=max_sent_length,
                    batch_size=batch_size,
                    num_classes=2,
                    sequence_length=sequence_length, 
                    vocab_size=word2vec_matrix.shape[0],
                    embedding_size=word2vec_matrix.shape[1],
                    filter_sizes=[1, 2, 3],
                    conv_output_len=50,
                    num_rnn_units=128)

TF_SAVEDIR = "./model/temp"
checkpoint_filename = os.path.join(TF_SAVEDIR, "model")
trained_filename = os.path.join(TF_SAVEDIR, "model_trained")

In [ ]:
reload(CNNRNN)

# Will print status every this many seconds
print_interval = 5

lm = CNNRNN.CNNRNN(**model_params)
lm.BuildCoreGraph()
lm.BuildTrainGraph()

# Explicitly add global initializer and variable saver to LM graph
with lm.graph.as_default():
    initializer = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
# Clear old log directory
shutil.rmtree(TF_SAVEDIR, ignore_errors=True)
if not os.path.isdir(TF_SAVEDIR):
    os.makedirs(TF_SAVEDIR)

with tf.Session(graph=lm.graph) as session:
    # Seed RNG for repeatability
    tf.set_random_seed(42)

    session.run(initializer, feed_dict={lm.embedding_: word2vec_matrix})
    
    for epoch in range(1,num_epochs+1):
        t0_epoch = time.time()
        bi = batch_generator(train_x, train_y, batch_size)
        
        print("[epoch {:d}] Starting epoch {:d}".format(epoch, epoch))

        # Run a training epoch.
        run_epoch(lm, session, bi, train=True, verbose=True, tick_s=10, learning_rate=learning_rate)
        
        print("[epoch {:d}] Completed in {:s}".format(epoch, pretty_timedelta(since=t0_epoch)))
    
        # Save a checkpoint
        saver.save(session, checkpoint_filename, global_step=epoch)
    
    # Save final model
    saver.save(session, trained_filename)

In [54]:
lm = CNNRNN.CNNRNN(**model_params)
lm.BuildCoreGraph()

with lm.graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=lm.graph) as session:
    
    # Load the trained model
    saver.restore(session, trained_filename)

    score_dataset(lm, session, train_x, train_y, batch_size=200, name="Train")
    print()
    score_dataset(lm, session, dev_x, dev_y, batch_size=200, name="Dev")
    print()
    score_dataset(lm, session, test_x, test_y, batch_size=200, name="Test")

INFO:tensorflow:Restoring parameters from ./model/temp/model_trained
[batch 6]: loss = 0.640, accuracy = 0.649
[batch 14]: loss = 0.645, accuracy = 0.645
[batch 22]: loss = 0.646, accuracy = 0.650
[batch 30]: loss = 0.648, accuracy = 0.651
[batch 38]: loss = 0.651, accuracy = 0.648
[batch 45]: loss = 0.651, accuracy = 0.648
[batch 53]: loss = 0.652, accuracy = 0.648
[batch 61]: loss = 0.652, accuracy = 0.649
[batch 69]: loss = 0.650, accuracy = 0.652
[batch 77]: loss = 0.646, accuracy = 0.653
Train: avg. loss: 0.643  (perplexity: 1.90), accuracy: 0.656

[batch 6]: loss = 0.652, accuracy = 0.663
Dev: avg. loss: 0.656  (perplexity: 1.93), accuracy: 0.657

[batch 6]: loss = 0.668, accuracy = 0.640
Test: avg. loss: 0.664  (perplexity: 1.94), accuracy: 0.647


In [126]:
# Model parameters
model_params = dict(max_sent_length=max_sent_length,
                    batch_size=1,
                    num_classes=2,
                    sequence_length=sequence_length, 
                    vocab_size=word2vec_matrix.shape[0],
                    embedding_size=word2vec_matrix.shape[1],
                    filter_sizes=[1, 2, 3],
                    conv_output_len=50,
                    num_rnn_units=128)

lm = CNNRNN.CNNRNN(**model_params)
lm.BuildCoreGraph()

with lm.graph.as_default():
    saver = tf.train.Saver()

preds = []

with tf.Session(graph=lm.graph) as session:

    # Load the trained model
    saver.restore(session, trained_filename)

    for i in range(len(train_x)):
        pred = session.run(lm.logits_, feed_dict={lm.input_x_: np.reshape(train_x[i], [1, max_sent_length, sequence_length])})
        preds.append(np.exp(pred[0][1])/sum(np.exp(pred[0])))
        if i % 100 == 0:
            print(i)

INFO:tensorflow:Restoring parameters from ./model/temp/model_trained
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000


In [127]:
out, _ = pd.cut([int(x) for x in train_data.year_filed], 10, retbins=True)
bins = pd.cut([int(x) for x in train_data.year_filed], 10, labels=False)

preds_df = pd.DataFrame({'bins': bins, 'log_prob': np.log(preds)})

agg_df = preds_df.groupby('bins').agg(lambda x: -np.mean(x.log_prob))
agg_df.index = out.categories
agg_df.columns = ['cross_entropy_loss']

agg_df

,cross_entropy_loss
"(1791.774, 1814.6]",1.061668
"(1814.6, 1837.2]",1.015342
"(1837.2, 1859.8]",1.049687
"(1859.8, 1882.4]",1.074655
"(1882.4, 1905.0]",1.050141
"(1905.0, 1927.6]",1.068226
"(1927.6, 1950.2]",1.005111
"(1950.2, 1972.8]",0.917637
"(1972.8, 1995.4]",0.821081
"(1995.4, 2018.0]",0.845767
